# Языковые модели на n-граммах

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* https://www.nltk.org/api/nltk.util.html
* https://web.stanford.edu/~jurafsky/slp3/3.pdf
* https://www.youtube.com/watch?v=QGT6XTeA3YQ

In [1]:
import pandas as pd
import nltk
import re

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
pip install pymorphy3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 56.7 MB/s eta 0:00:00


## Задачи для совместного разбора

1\. Выделите из текста n-граммы.

In [4]:
text = """Вода это жидкость которая имеет свойство быть водой.
Вода состоит из молекул, которые выглядят как вода"""

In [5]:
import nltk

from nltk import word_tokenize, sent_tokenize
from nltk import RegexpTokenizer
from nltk.util import ngrams

nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [6]:
sents = sent_tokenize(text)
tokenizer = RegexpTokenizer("\w+")

unigrams = []
bigrams = []

for sent in sents:
  tokens = tokenizer.tokenize(sent.lower())
  # unigrams.extend(tokens)
  unigrams.extend(ngrams(tokens, n=1))
  bigrams.extend(ngrams(tokens, n=2))

In [7]:
bigrams[:5], unigrams[:5]

([('вода', 'это'),
  ('это', 'жидкость'),
  ('жидкость', 'которая'),
  ('которая', 'имеет'),
  ('имеет', 'свойство')],
 [('вода',), ('это',), ('жидкость',), ('которая',), ('имеет',)])

In [8]:
example_dict = {}
example_dict[('вода',)] = 1

2. Рассчитайте вероятности  $P(вода)$, $P(это|вода)$, $P(состоит|вода)$.

$$P(w_i) = \frac{C(w_i)}{N}$$
$$P(w_i|w_{i-1})=\frac{C(w_{i-1} w_i)}{C(w_{i-1})}$$

In [9]:
uni = ('вода',)
unigrams.count(uni), len(unigrams), unigrams.count(uni)/len(unigrams)

(3, 16, 0.1875)

In [10]:
[(prev, next) for (prev, next) in bigrams if prev == "вода"]

[('вода', 'это'), ('вода', 'состоит')]

P(это|вода) = 1 / 3

P(состоит|вода) = 1 / 3

3. Рассчитайте вероятности  $P_L(вода)$, $P_L(это|вода)$, $P_L(состоит|вода)$.

$$P_L(w_i) = \frac{C(w_i)+1}{N+V}$$, $$P_L(w_i|w_{i-1})=\frac{C(w_{i-1} w_i)+1}{C(w_{i-1})+V}$$

## Задачи для самостоятельного решения

<p class="task" id="1"></p>

1\. Считайте файл `data/moya-semia/Лучше кошки зверя нет 2.csv`. Получите список предложений из сообщений. Приведите предложения к нижнему регистру и удалите все символы, кроме букв и пробелов. Получите список слов (униграмм) и биграмм.

- [ ] Проверено на семинаре

In [11]:
import pandas as pd
import re
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.util import bigrams


In [12]:
# Загрузите файл
file_path = "/content/drive/MyDrive/NLP/1-2/data/Лучше кошки зверя нет 2.csv"
df = pd.read_csv(file_path, usecols=[1], names=['text'])
df

,text
0,Завтра мои котейки идут к врачу. А 21-го жду и...
1,Котя скучает по мужу. Со вторника спит на его ...
2,"Получилось так, что мне пришлось ""отдать"" мою ..."
3,Лучше всего для начала спросить об этом у люде...
4,"что волонтеры принесут, то и насыпалиЗначит, к..."
...,...
1579,Надо на ночь сыпать больше корма.Я своим насып...
1580,А вот когда реально голодные - кусаются. Могут...
1581,И нет разницы в эмоциях человека и кота. ...
1582,Который деньТоже запал.Если не пристроили - на...


In [13]:
tokenizer = RegexpTokenizer(r'w+')

In [14]:
sentences = []
words = []

for text in df['text'].dropna():
    text_sentences = sent_tokenize(text)
    text_sentences = [re.sub(r'[^а-яА-Я ]', ' ', s.lower()) for s in text_sentences]
    sentences.extend(text_sentences)

    for sent in text_sentences:
        tokens = word_tokenize(sent)
        words.extend(tokens)

bigrams_list = list(bigrams(words))

In [15]:
text = " ".join(df.values.flatten()).lower().replace('\xa0', ' ')

In [16]:
sentences = sent_tokenize(text)
tokenizer = RegexpTokenizer(r"\w+")
sentences[:5]

['завтра мои котейки идут к врачу.',
 'а 21-го жду их домой.',
 'прошу помощи.',
 'чем кормить лучше (корм).',
 'какой наполнитель использовать.']

In [17]:
words = []

In [18]:
for sent in sentences:
    tokens = tokenizer.tokenize(sent)
    words.extend(tokens)

bigrams_list = list(bigrams(words))


In [19]:
len(words)

175650

<p class="task" id="2"></p>

2\. Получите распределение вероятностей для униграм $P(w_i) = \frac{C(w_i)}{N}$, где $N$ - количество униграм, $C(w_i)$ - частота использования токена $w_i$. Получите распределение условных вероятностей для биграмм $P(w_i|w_{i-1})=\frac{C(w_{i-1} w_i)}{C(w_{i-1})}$ ($C(w_{i-1} w_i)$ - частота использования словосочетания $w_{i-1}w_i$).

- [ ] Проверено на семинаре

In [20]:
from collections import Counter

In [21]:

N = len(words)
P_u= [(i, g, g/N) for i, g in list(Counter(words).items())]
P_u.sort(key=lambda x: -x[1])
P_u[:10]

[('и', 6471, 0.03684030742954739),
 ('не', 4428, 0.025209222886421863),
 ('в', 4216, 0.02400227725590663),
 ('на', 3912, 0.022271562766865927),
 ('с', 2358, 0.013424423569598633),
 ('я', 2288, 0.013025903785937945),
 ('что', 2276, 0.01295758610873897),
 ('а', 2154, 0.012263023057216054),
 ('у', 1689, 0.009615713065755765),
 ('как', 1442, 0.008209507543410191)]

In [22]:
# N = len(words)
# P_u = [(i, words.count(i), words.count(i) / N) for i in set(words)]
# P_u.sort(key=lambda x: -x[1])

In [23]:
P_u[:10]

[('и', 6471, 0.03684030742954739),
 ('не', 4428, 0.025209222886421863),
 ('в', 4216, 0.02400227725590663),
 ('на', 3912, 0.022271562766865927),
 ('с', 2358, 0.013424423569598633),
 ('я', 2288, 0.013025903785937945),
 ('что', 2276, 0.01295758610873897),
 ('а', 2154, 0.012263023057216054),
 ('у', 1689, 0.009615713065755765),
 ('как', 1442, 0.008209507543410191)]

In [24]:
len(words), len(bigrams_list), len(P_u)

(175650, 175649, 28593)

Чтобы долго не перебирать все возьмем топ 100 популярных слов и найдем распределение условных вероятностей для них

In [25]:
from collections import defaultdict

In [26]:
bi = defaultdict(int)

top100 = P_u[:100]
top_list = [i[0] for i in top100]
top100[-1], top_list[:5]

(('можно', 167, 0.000950754341019072), ['и', 'не', 'в', 'на', 'с'])

In [27]:
for (prev, next) in bigrams_list:
    if prev in top_list:
        bi[(prev, next)] += 1
len(bi)

31993

In [28]:
top100 = dict([(i, g) for i, g, _ in top100])
top100['и']

6471

In [29]:
P_b = defaultdict(list)
for (prev, next), v in bi.items():
    P_b[prev].append([(next, v, v/top100[prev])])

In [30]:
P_b['и'][:10]

[[('вопит', 5, 0.0007726781023025807)],
 [('продукты', 2, 0.0003090712409210323)],
 [('уселся', 3, 0.0004636068613815484)],
 [('сидит', 20, 0.003090712409210323)],
 [('упирается', 2, 0.0003090712409210323)],
 [('сидел', 6, 0.0009272137227630969)],
 [('свалил', 4, 0.0006181424818420646)],
 [('ехидненько', 2, 0.0003090712409210323)],
 [('плохо', 3, 0.0004636068613815484)],
 [('насыпали', 2, 0.0003090712409210323)]]

Работает, повторим на всем...

In [31]:
bi = defaultdict(int)
words_set = list(set(words))

In [32]:
for (prev, next) in bigrams_list:
    bi[(prev, next)] += 1
len(bi)

116524

In [33]:
tir_list = dict([(i, g) for i, g, _ in P_u])
tir_list['и']

6471

In [34]:
P_b = defaultdict(list)
for (prev, next), v in bi.items():
    P_b[prev].append([next, v, v/tir_list[prev]])

In [35]:
len(P_b), len(P_b['и']), P_b['и'][0]

(28593, 3088, ['вопит', 5, 0.0007726781023025807])

<p class="task" id="3"></p>

3\.Воспользовавшись полученными вероятностями, сгенерируйте текст длиной не более 20 символов, начинающийся с токена "мой". При генерации текста выбирайте слово с наибольшей вероятностью соответствующего биграмма. Выведите полученный текст на экран.

- [ ] Проверено на семинаре

In [36]:
text = ['мой']
for i in range(19):
    x = sorted(P_b[text[-1]], key=lambda x: x[2])[-1]

    text.append(x[0])

' '.join(text)

'мой кот в доме это не знаю что то что то что то что то что то что то что'

In [37]:
a = sorted(P_b['что'], key=lambda x: -x[2])
a[:20]

[['то', 130, 0.05711775043936731],
 ['я', 89, 0.03910369068541301],
 ['это', 82, 0.03602811950790861],
 ['он', 61, 0.02680140597539543],
 ['у', 57, 0.025043936731107205],
 ['не', 55, 0.024165202108963092],
 ['она', 52, 0.022847100175746926],
 ['делать', 45, 0.01977152899824253],
 ['в', 40, 0.01757469244288225],
 ['ли', 37, 0.01625659050966608],
 ['кот', 37, 0.01625659050966608],
 ['и', 30, 0.013181019332161687],
 ['вы', 29, 0.012741652021089631],
 ['на', 25, 0.010984182776801407],
 ['с', 24, 0.01054481546572935],
 ['за', 22, 0.009666080843585237],
 ['они', 22, 0.009666080843585237],
 ['мы', 21, 0.00922671353251318],
 ['кошка', 21, 0.00922671353251318],
 ['там', 20, 0.008787346221441126]]

In [38]:
tir_list['что']

2276

In [39]:
89/2101

0.04236078058067587

In [40]:
len(words)

175650

<p class="task" id="4"></p>

4\.Воспользовавшись полученными вероятностями, сгенерируйте текст длиной не более 20 символов, начинающийся с токена "мой". При генерации текста выбирайте слово пропорционально вероятностям соответствующих биграммов. Выведите полученный текст на экран.

- [ ] Проверено на семинаре

In [41]:
import random

def generate_text(start_word, max_length=20):
    text = [start_word]  # Начинаем с заданного слова
    current_word = start_word

    for i in range(max_length):

        # Получаем список возможных следующих слов и их вероятностей
        next_words = P_b[current_word]
        words, counts, probabilities = zip(*[(x[0], x[1], x[2]) for x in next_words])

        # Выбираем следующее слово пропорционально вероятностям
        next_word = random.choices(words, weights=probabilities, k=1)[0]
        text.append(next_word)
        current_word = next_word

    return " ".join(text)

# Генерация текста, начиная с "мой"
generated_text = generate_text("мой", max_length=20)
generated_text

'мой укол сотрудники слышали все из предков затесался блю табби полоски как с кошандрой вернулась в ванной комнате половина в этом'

<p class="task" id="5"></p>

5\. Получите распределение вероятностей для униграм, воспользовавшись сглаживанием Лапласа: $P_L(w_i) = \frac{C(w_i)+1}{N+V}$, где $V$ - количество уникальных униграмм. Получите распределение условных вероятностей для биграмм $P_L(w_i|w_{i-1})=\frac{C(w_{i-1} w_i)+1}{C(w_{i-1})+V}$

- [ ] Проверено на семинаре

In [42]:
len(words), len(words_set)

(175650, 28593)

In [43]:
N, V = len(words), len(words_set)

In [44]:
P_u[:5]

[('и', 6471, 0.03684030742954739),
 ('не', 4428, 0.025209222886421863),
 ('в', 4216, 0.02400227725590663),
 ('на', 3912, 0.022271562766865927),
 ('с', 2358, 0.013424423569598633)]

In [45]:
N = len(words)

Pl_u= [(i, g, g/(N+g)) for i, g in list(Counter(words).items())]
Pl_u.sort(key=lambda x: -x[1])
Pl_u[:10]

[('и', 6471, 0.03553132258223928),
 ('не', 4428, 0.02458934461733249),
 ('в', 4216, 0.02343967175564031),
 ('на', 3912, 0.02178634677715775),
 ('с', 2358, 0.013246595658622084),
 ('я', 2288, 0.012858411356764715),
 ('что', 2276, 0.012791834807729056),
 ('а', 2154, 0.012114463116690288),
 ('у', 1689, 0.009524131747669718),
 ('как', 1442, 0.008142660312154135)]

In [46]:
P_b['и'][:5]

[['вопит', 5, 0.0007726781023025807],
 ['продукты', 2, 0.0003090712409210323],
 ['уселся', 3, 0.0004636068613815484],
 ['сидит', 20, 0.003090712409210323],
 ['упирается', 2, 0.0003090712409210323]]

In [47]:
Pl_b = defaultdict(list)
for (prev, next), v in bi.items():
    Pl_b[prev].append([next, v, v/(tir_list[prev]+V)])

In [48]:
Pl_b['и'][:5]

[['вопит', 5, 0.0001425963951631303],
 ['продукты', 2, 5.703855806525211e-05],
 ['уселся', 3, 8.555783709787816e-05],
 ['сидит', 20, 0.0005703855806525211],
 ['упирается', 2, 5.703855806525211e-05]]

<p class="task" id="6"></p>

6\.Воспользовавшись полученными после сглаживания вероятностями, сгенерируйте текст длиной не более 20 символов, начинающийся с токена "мой". При генерации текста выбирайте слово пропорционально вероятностям соответствующих биграммов. Выведите полученный текст на экран.

- [ ] Проверено на семинаре

In [49]:
import random

def generate_text(start_word, max_length=20):
    text = [start_word]  # Начинаем с заданного слова
    current_word = start_word

    for i in range(max_length):

        # Получаем список возможных следующих слов и их вероятностей
        next_words = Pl_b[current_word]
        words, counts, probabilities = zip(*[(x[0], x[1], x[2]) for x in next_words])

        # Выбираем следующее слово пропорционально вероятностям
        next_word = random.choices(words, weights=probabilities, k=1)[0]
        text.append(next_word)
        current_word = next_word

    return " ".join(text)

# Генерация текста, начиная с "мой"
generated_text = generate_text("мой", max_length=20)
generated_text

'мой сон не надолго не спит вместе с асей а когда я с урчанием он специально в кухне перед ее несколько'

<p class="task" id="7"></p>

7\. Рассчитайте перплексию для текста "Котя пришел домой с хромой лапой" для четырех моделей: на 1/2-граммах и с/без использования сглаживания Лапласа. Сведите результат в таблицу. Повторите вычисления для текста "После пар я поеду кормить своего кота", используя доступные модели.

$Perplexity(W) = P(w_1w_2...w_N)^{-\frac{1}{N}}$

Для модели на униграммах $P(w_1w_2...w_N) = \Pi_{i=1}^{N}{P(w_i)}$

Для модели на биграммах $P(w_1w_2...w_N) = \Pi_{i=1}^{N}{P(w_i|w_{i-1})}$

- [ ] Проверено на семинаре

In [50]:
from decimal import Decimal

In [51]:
P_u[:5]

[('и', 6471, 0.03684030742954739),
 ('не', 4428, 0.025209222886421863),
 ('в', 4216, 0.02400227725590663),
 ('на', 3912, 0.022271562766865927),
 ('с', 2358, 0.013424423569598633)]

In [52]:
P_b['и'][:5]

[['вопит', 5, 0.0007726781023025807],
 ['продукты', 2, 0.0003090712409210323],
 ['уселся', 3, 0.0004636068613815484],
 ['сидит', 20, 0.003090712409210323],
 ['упирается', 2, 0.0003090712409210323]]

In [53]:
sent1 = "Котя пришел домой с хромой лапой".lower()

In [54]:
sent2 = "После пар я поеду кормить своего кота".lower()

In [55]:
def perplexity1(text):
    text = word_tokenize(text)
    N = -1/len(text)
    first_word = text[0]
    Per = 1
    for i in range(len(text)):
        checker = Per
        for g in P_u:
            if text[i] == g[0]:
                Per *= g[2] ** N
                # print('Сделал', g[0])
        if checker == Per:
            return 'не найдено слово: '+ '!'+text[i]+'!'
    return Per


In [56]:
perplexity1(sent1), perplexity1(sent2)

(1378.766380084801, 3932.132958585642)

In [57]:
N_min = len(Pl_b)
N_min

28593

In [58]:
def perplexity2(text):
    text = word_tokenize(text)
    N = -1/len(text)
    first_word = text[0]
    Per = 1
    for i in range(len(text)-1):
        checker = Per
        for g in P_b[text[i]]:
            if text[i+1] == g[0]:
                Per *= g[2] ** N
                # print(g[0])
        if checker == Per:
            return 'не обрабатывается'
            # print(text[i], 'заменен на минимальное число')
    return Per


In [59]:
perplexity2(sent1), perplexity2(sent2)

(15.131545172104229, 'не обрабатывается')

In [60]:
len(words_set)

28593

Не уверен что тут нужно добавлять V (уникальные униграммы), но без них вылетают какие-то так называемые "миллионы"  

    N = -1/(len(text)+len(words_set))


In [61]:
def perplexity3(text):
    text = word_tokenize(text)
    N = -1/len(text)
    first_word = text[0]
    Per = 1
    for i in range(len(text)):
        checker = Per
        for g in Pl_u:
            if text[i] == g[0]:
                Per *= g[2] ** N
                # print('Сделал', g[0])
        if checker == Per:
            return 'не найдено слово: '+ '!'+text[i]+'!'
    return Per


In [62]:
perplexity3(sent1), perplexity3(sent2)

(1382.6481738530372, 3941.574232626599)

In [66]:
def perplexity4(text):
    text = word_tokenize(text)
    N = -1/len(text)
    first_word = text[0]
    Per = 1
    for i in range(len(text)-1):
        checker = Per
        for g in Pl_b[text[i]]:
            if text[i+1] == g[0]:
                Per *= g[2] ** N
                # print(g[0])
        if checker == Per:
            return 'не обрабатывается'
            # print(text[i], 'заменен на минимальное число')
    return Per


In [67]:
perplexity4(sent1), perplexity4(sent2)

(1357.8807295073038, 'не обрабатывается')

In [68]:
pd.DataFrame({
    'unigram': [perplexity1(sent1), perplexity1(sent2)],
    'bigram': [perplexity2(sent1), perplexity2(sent2)],
    'unigram_laplas': [perplexity3(sent1), perplexity3(sent2)],
    'bigram_laplas': [perplexity4(sent1), perplexity4(sent2)]
}, index=['текст №1', 'текст №2'])

,unigram,bigram,unigram_laplas,bigram_laplas
текст №1,1378.766380,15.131545,1382.648174,1357.88073
текст №2,3932.132959,не обрабатывается,3941.574233,не обрабатывается
